In [59]:
%matplotlib inline
import tensorflow as tf
import numpy as np

In [8]:
a = tf.constant(2, dtype=tf.int32)
b = tf.constant(5, dtype=tf.int32)

In [9]:
with tf.Session() as sess:
    print("a: {}, b: {}".format(sess.run(a), sess.run(b)))
    print("addition: {}".format(sess.run(a+b)))

a: 2, b: 5
addition: 7


In [11]:
a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)
mult = tf.multiply(a, b)
with tf.Session() as sess:
    print(sess.run(mult, feed_dict={a: 2, b:3}))


6


In [22]:
from tensorflow.examples.tutorials.mnist import input_data

In [23]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [24]:
Xtr, Ytr = mnist.train.next_batch(5000)

In [27]:
Xte,Yte = mnist.test.next_batch(200)

In [29]:
xtr = tf.placeholder("float", [None, 784])
xte = tf.placeholder("float", [784])

In [50]:
distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), axis=1)
pred = tf.arg_min(distance, 0)

In [51]:
tf.global_variables_initializer()

<tf.Operation 'init_1' type=NoOp>

In [52]:
tf.GraphKeys.GLOBAL_VARIABLES

'variables'

In [54]:
with tf.Session() as sess:
    a = sess.run(distance, feed_dict={xtr:Xtr, xte:Xte[0, :]})
    print(a)
    b = sess.run(pred, feed_dict={distance:a})
    print(b)

[ 120.92158508  115.52942657   95.49412537 ...,  124.69413757  104.44314575
   74.47451782]
4444


### Logistic Regression

In [60]:
#Model parameters
learning_rate = 0.1
training_epochs = 25
batch_size = 100


## Data
## num_objects * 784 features
x = tf.placeholder(tf.float32, [None, 784])
## prob for each class
y = tf.placeholder(tf.float32, [None, 10])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

pred = tf.nn.softmax(tf.matmul(x, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [71]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0
        total_num_batch = int(mnist.train.num_examples/batch_size)
        for bi in range(total_num_batch):
            b_x, b_y = mnist.train.next_batch(batch_size)
            
            _, c = sess.run([optimizer, cost], feed_dict={x: b_x, y: b_y})
            avg_cost += c/total_num_batch
        
        #print("Epoch:{}, cost={}", epoch, avg_cost)
        #print("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))

In [64]:
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

In [65]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#### UniVariate Hawkes 

## $\lambda^*(t) = \mu + \alpha \sum_{i=0}^t \exp^{-\beta (t - t_i)}$

## $L(<0,..., T>| \alpha , \beta, \mu) = \{\pi_{i=0}^T \lambda^*(t) \} \exp(\int_{0}^{T} -\lambda^*(t) dt)$

## $Loglikelihood = \sum_{i=0}^{T} \lambda^*(t_i) - \mu T + \frac{\alpha}{\beta}\sum_{i<T}\{ \exp\{-\beta (T-t_i)\} - 1\} $



In [171]:
## Assuming number of users is 32
num_users = 1

## All parameters are column vectors with size equal to number of users

alpha = tf.Variable(tf.zeros([num_users, 1]))
beta = tf.Variable(tf.ones([num_users, 1]))
mu = tf.Variable(tf.zeros([num_users, 1]))
test = tf.zeros_like(arrivals)

In [187]:
## Create place holders for time data, assume maximum number of time-steps/events is 100
T_max = 5000
arrivals = tf.placeholder(tf.float32, [None, T_max])
mask = tf.placeholder(tf.float32, [None, T_max])
ais = tf.placeholder(tf.float32,[None, T_max])

In [188]:
def diff(a, n=1, axis=-1):
    """copied shamelessly from `http://stackoverflow.com/questions/42609618/tensorflow-equivalent-to-numpy-diff anwer by Frank Dernoncourt`"""
    if n == 0:
        return a
    if n < 0:
        raise ValueError(
            "order must be non-negative but got " + repr(n))
    nd = len(a.shape)
    slice1 = [slice(None)]*nd
    slice2 = [slice(None)]*nd
    slice1[axis] = slice(1, None)
    slice2[axis] = slice(None, -1)
    slice1 = tuple(slice1)
    slice2 = tuple(slice2)
    if n > 1:
        return diff(a[slice1]-a[slice2], n-1, axis=axis)
    else:
        return a[slice1]-a[slice2]

In [189]:
def loglikelihood():
    tdiff = diff(arrivals) * mask[:, 1:]
    A_i = tf.zeros_like(arrivals)
    for i in range(tdiff.shape[0]):
        A_i[:, i + 1] = tdiff[:, i] * (1 + A[:, i ])
    
    part1 = tf.reduce_sum(tf.log(mu + alpha * A_i), axis=1)
        
    part2 = mu * T_max
    part3 = (alpha / beta)* tf.reduce_sum(tf.exp(-beta * ((T_max - arrivals)*mask)) - tf.constant(1.0, dtype=tf.float32))
    
    return part1 - part2 + part3

def cost_func():
    return -loglikelihood()
    

In [190]:
import pandas as pd

In [191]:
df = pd.read_csv("/home/sathappan/workspace/time2event/hawkes/data/all_trades.csv")
df.t = pd.to_datetime(df.t)
pp = (df.t - df.t[0]).apply(lambda x: x/np.timedelta64(1, 'm')).value_counts()
events = []
e = 1.667e-6
for t, cnt in pp.iteritems():
    base_time = t
    for i in range(cnt):
        events.append(base_time + e*i)

events = np.array(sorted(events))
empirical = df.t
empirical_counts = empirical.value_counts().resample(rule='1min').sum()

In [192]:
events = events[None, :]

In [193]:
events.shape

(1, 5000)

In [194]:
init = tf.global_variables_initializer()

In [175]:
with tf.Session() as sess:
    sess.run(init)
    negloglik = cost_func()
    sess.run(negloglik, feed_dict={arrivals: events, mask: np.ones_like(events)})

TypeError: __index__ returned non-int (type NoneType)

In [176]:
len(np.zeros((5,3)))

5

In [177]:
test_session = tf.Session()
test_session.run(init)
b = test_session.run(diff(arrivals) *mask[:, 1:], feed_dict={arrivals: events, mask: np.ones_like(events)})

In [178]:
b.shape

(1, 4999)

In [195]:
def p2():
    tdiff = diff(arrivals) * mask[:, 1:]
    #ei = np.zeros_like(arrivals)
    for i in range(4999):
        ais[:, i + 1] += tdiff[:, i] * (1 + ais[:, i ])
    return ei

In [196]:
test_session.run(p2(), feed_dict={arrivals: events, mask: np.ones_like(events)})

TypeError: 'Tensor' object does not support item assignment

In [154]:
alpha.get_shape()[1]

Dimension(1)